In [5]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [ ]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

In [13]:
train_dataset_path = "images/train"
validation_dataset_path = "images/validation"

In [14]:
train_data = object_detector.Dataset.from_pascal_voc_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_pascal_voc_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  150
validation_data size:  150


In [15]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [16]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


18/18 [==============================] - 76s 863ms/step - total_loss: 5.0099 - cls_loss: 4.5527 - box_loss: 0.0079 - model_loss: 4.9470 - val_total_loss: 1.7896 - val_cls_loss: 1.2711 - val_box_loss: 0.0091 - val_model_loss: 1.7267
Epoch 2/30
18/18 [==============================] - 4s 237ms/step - total_loss: 1.4462 - cls_loss: 1.1338 - box_loss: 0.0050 - model_loss: 1.3833 - val_total_loss: 1.4460 - val_cls_loss: 1.1267 - val_box_loss: 0.0051 - val_model_loss: 1.3831
Epoch 3/30
18/18 [==============================] - 5s 256ms/step - total_loss: 1.1251 - cls_loss: 0.8872 - box_loss: 0.0035 - model_loss: 1.0622 - val_total_loss: 1.1579 - val_cls_loss: 0.8876 - val_box_loss: 0.0041 - val_model_loss: 1.0951
Epoch 4/30
18/18 [==============================] - 4s 234ms/step - total_loss: 0.8135 - cls_loss: 0.6042 - box_loss: 0.0029 - model_loss: 0.7506 - val_total_loss: 0.9120 - val_cls_loss: 0.6552 - val_box_loss: 0.0039 - val_model_loss: 0.8492
Epoch 5/30
18/18 [========================

In [17]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

38/38 [==============================] - 7s 125ms/step - total_loss: 0.6050 - cls_loss: 0.3547 - box_loss: 0.0037 - model_loss: 0.5421
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.499
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.935
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.478
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.497
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.550
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.583
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.585
 Average Recall     (AR) @[

In [18]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  metadata.json  model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>